# Cortex Compliance AI - Fine-Tuning Notebook

Fine-tunes Mistral-7B for Russian compliance document generation.

## Requirements:
- **Google Colab (Recommended)**: Free T4 GPU, pre-configured
- **Local**: NVIDIA GPU with 8GB+ VRAM, CUDA 11.8+, Linux recommended

## Quick Start (Colab):
1. Runtime → Change runtime type → **T4 GPU**
2. Run Step 1, then Runtime → Restart runtime
3. Run Steps 2-9

## Why Colab?
- 217 training examples is small but sufficient for LoRA
- Fine-tuning 7B model needs ~10GB VRAM (even quantized)
- Colab provides free GPU access

In [ ]:
# Step 1: Install dependencies
# After running this cell: Runtime → Restart runtime, then run from Step 2

import subprocess
import sys

# Uninstall conflicting versions quietly
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "transformers", "accelerate", "peft", "bitsandbytes", "triton"], 
               capture_output=True)

# Install compatible versions + triton for GPU optimization
!pip install -q transformers==4.44.0 accelerate==0.33.0 peft==0.12.0 bitsandbytes==0.43.1
!pip install -q datasets huggingface_hub trl sentencepiece protobuf
!pip install -q triton  # GPU optimization

print("\n✅ Packages installed!")
print("👉 Now go to: Runtime → Restart runtime")
print("👉 Then run from Step 2 (skip this cell)")

In [ ]:
# Step 2: Login to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Step 3: Load Training Data - 217 Russian Business Document Templates
# Includes: Contracts, Corporate Docs, Financial, HR, Legal, Tax, Industry, Specialized and more!

import json

# Download training data from GitHub (217 examples from 265 templates)
!wget -q https://raw.githubusercontent.com/maanisingh/cortex-compliance-ai/main/combined_training_data.jsonl -O training_data.jsonl

# Load training data
TRAINING_DATA = []
with open('training_data.jsonl', 'r') as f:
    for line in f:
        TRAINING_DATA.append(json.loads(line))

print(f"Loaded {len(TRAINING_DATA)} training examples")
print(f"\nSample categories:")
for i, item in enumerate(TRAINING_DATA[:5]):
    print(f"  {i+1}. {item['instruction'][:60]}...")

In [ ]:
# Step 4: Prepare dataset
from datasets import Dataset

def format_prompt(example):
    return {"text": f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"}

dataset = Dataset.from_list(TRAINING_DATA)
dataset = dataset.map(format_prompt)
print(dataset)

In [ ]:
# Step 5: Load model with 4-bit quantization
import warnings
warnings.filterwarnings("ignore")

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)
print(f"✅ Model loaded: {MODEL_NAME}")

In [ ]:
# Step 6: Configure LoRA
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Step 7: Train using Trainer
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)

training_args = TrainingArguments(
    output_dir="./cortex-compliance-ai",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    optim="paged_adamw_8bit",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")

In [ ]:
# Step 8: Save to Hugging Face Hub
from huggingface_hub import whoami

# Get your HuggingFace username automatically
hf_user = whoami()["name"]
MODEL_REPO = f"{hf_user}/cortex-compliance-ai"

print(f"Uploading to: {MODEL_REPO}")
model.push_to_hub(MODEL_REPO)
tokenizer.push_to_hub(MODEL_REPO)
print(f"Model saved to: https://huggingface.co/{MODEL_REPO}")

In [ ]:
# Step 9: Test the model
test_prompt = "### Instruction:\nGenerate a Personal Data Processing Policy for ООО Тест (INN: 1234567890)\n\n### Response:\n"
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=300,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Done!

Your model is now available at: https://huggingface.co/maaninder/cortex-compliance-ai

The Cortex GRC backend is configured to use this model for AI document generation.